In [2]:

import xarray as xr
import numpy as np
from analysisFuncs import ens_list,templates,applyFunc,plot_map
from glob import glob
import dask
dask.config.set({"array.slicing.split_large_chunks": True})
import xesmf as xe
import matplotlib.pyplot as plt
from dask_jobqueue import PBSCluster
import dask
from dask.distributed import Client
import os
import matplotlib as mpl
import cartopy.crs as ccrs

cluster = PBSCluster(
    queue="casper",
    walltime="2:00:00",
    project="P48500028",
    memory="20GB",
    cores=1,
    processes=1,
)

cluster.scale(10)


/glade/work/shartke/conda-envs/samEnv/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42825 instead
  f"Port {expected} is already in use.\n"


In [ ]:

vardict = {'cesmlens2':'TREFHT',
          'canesm5':'tas',
          'ecearth3':'tas'}

gcm_longnames = {'cesmlens2':'CESM2',
                 'canesm5':'CanESM5',
                 'ecearth3':'EC-Earth3'}

lim_ds = []

letters = ('a.','b.','c.','d.','e.','f.','g.','h.','i.')

fig = plt.figure(figsize=(16,10))
proj = ccrs.PlateCarree()
vmin=8;vmax=37.

i = 1
for gcm in ('cesmlens2','ecearth3','canesm5'):

    print(gcm)
    
    # Cycle through all GARDLENS ensemble members for this GCM and find the historical max at each grid cell
    max_temps = []
    for e in ens_list[gcm]:
        print(e)
        ds = xr.open_dataset(templates['GARD'].format(gcm=gcm,obs='gmet',ens=e,n=0,var='t_mean',pred='pALL',runtag='_AK')).sel(time=slice('1980','2014'))
        ds = ds.assign(y=ds.lat[:,0].values).assign(x=ds.lon[0,:].values-360).drop_vars(('lat','lon')).rename({'y':'lat','x':'lon'})
        max_temps.append(ds.quantile(.999,'time'))
    
    all_maxs = xr.concat(max_temps,dim='n_ens')
    gard_max = all_maxs.max('n_ens').drop('quantile')

    ax1 = fig.add_subplot(3,3,i,projection=proj)
    plot_map(gard_max.t_mean,ax=ax1,vmin=vmin,vmax=vmax,bottom=False,left=False,colorbar=False,cmap='plasma')
    if i==1: ax1.set_title('a. Maximum 1980-2014 t_mean from\nGARD LENS simulation')
    #ax1.annotate('GARD LENS (%s)'%gcm_longnames[gcm],xy=(-160.,53.),xycoords='data',fontsize=13)
    ax1.text(-0.04, 0.55,gcm_longnames[gcm], va='bottom', ha='center',rotation='vertical',
                 fontsize=18,rotation_mode='anchor',transform=ax1.transAxes,fontweight='bold')
    
    # Cycle through all ensemble members for this GCM and calculate the future trend in annual maximum t_mean at each grid cell
    max_trends = []
    gcm_files = xr.open_dataset('/glade/campaign/ral/hap/hartke/stat_files/%s_t_mean_stats_AK.nc'%gcm)
    
    for e in ens_list[gcm]:
        print(e)
        ds = gcm_files.sel(n_ens=gcm_files.ens==e).sel(year=slice('2015','2100'))['max'][0,:,:,:]
        max_trends.append(applyFunc(ds,'trend')*85) # multiply yearly trend by future period (85 years)
    
    gcm_max = xr.concat(max_trends,dim='n_ens').max('n_ens')
    # ds_out = xe.util.grid_2d(gard_max.lon[0].values.item(), gard_max.lon[-1].values.item(), 1/8., gard_max.lat[0].values.item(), gard_max.lat[-1].values.item(), -1/8.)
    regridder = xe.Regridder(gcm_max, gard_max, 'nearest_s2d')
    gcm_max_12k = regridder(gcm_max)

    ax1 = fig.add_subplot(3,3,i+1,projection=proj)
    plot_map(gcm_max_12k.drop('degree'),ax=ax1,vmin=0.,vmax=13.,bottom=False,left=False,colorbar=False,cmap='Oranges')
    if i==1: ax1.set_title('b. Maximum 2015-2100\nt_mean signal from GCM')
    ax1.text(-0.055, 0.52,'+', va='bottom', ha='center',rotation='vertical',fontsize=28,rotation_mode='anchor',transform=ax1.transAxes,fontweight='bold')
    
    tot_max = gard_max + gcm_max_12k.values
    tot_max['gcm'] = gcm

    ax1 = fig.add_subplot(3,3,i+2,projection=proj)
    plot_map(tot_max.t_mean,ax=ax1,vmin=vmin,vmax=vmax,bottom=False,left=False,colorbar=False,cmap='plasma')
    if i==1: ax1.set_title('c. t_mean limit for GARD LENS\n2015-2100 simulations')
    ax1.text(-0.09, 0.46,'=', va='bottom', ha='center',fontsize=28,rotation_mode='anchor',transform=ax1.transAxes,fontweight='bold')
    
    lim_ds.append(tot_max)

    i+=3


xloc = 0.06
for var in ((8.,37.),(0.,13.),(8.,37.)):
    #cmap='plasma' if xloc!=.42: 'Oranges'
    ax2 = fig.add_axes([xloc, -0.02, 0.22, 0.02])
    cb = mpl.colorbar.ColorbarBase(ax2, orientation='horizontal', 
                                   cmap='plasma' if xloc!=.395 else 'Oranges',
                                   norm=mpl.colors.Normalize(var[0], var[1]),  # vmax and vmin
                                   extend='both',
                                   label='t_mean [C]')
    xloc+=0.335

plt.tight_layout()
plt.savefig('figures/SupplementalFigure_Alaska_t_mean_limit.jpg',bbox_inches='tight',dpi=1200)
plt.show()


lim_ds = xr.concat(lim_ds,dim='gcm')

outfile = '/glade/campaign/ral/hap/hartke/stat_files/GARDLENS_Alaska_t_mean_limit.nc'
if os.path.exists(outfile):
    os.remove(outfile)
    
lim_ds.to_netcdf(outfile)
